In [8]:
%load_ext autoreload
%autoreload 2
import time
from tweepy import  TweepError
from requests.exceptions import Timeout, ConnectionError
import os
import tweepy
import json


# THE CELL BELOW CONTAINS THE FILENAME CONSTANTS THAT NEED TO BE CHANGED

In [ ]:
# this file is a line seperated list of tweet ids to retrieve
TWEET_ID_FILE = os.getcwd() + '/../data/data/tweet_ids_v1/irma_tweet_ids.txt'
# this file is a line seperated list of JSONs, the script extracts the set of users from this to filter the new tweets
USER_ID_FILE = os.getcwd() + "/../data/CrisisMMD_v1.0/json/hurricane_irma_final_data.json"

# the file to write the retrieved tweets too
OUTFILE = "irma_extras.json"
# a file to log any error errors
ERROR_FILE = "irma_extras_spill.json"


In [9]:
# get list of tweet id's to look up
tweet_ids = []
with open(TWEET_ID_FILE) as fin:
    for line in fin.readlines()[1:]:
        tweet_id = int(line)
        tweet_ids.append(tweet_id)

In [10]:
# get list of user id's to filter by
user_ids = set()
with open(USER_ID_FILE) as fin:
    for line in fin:
        json_data = json.loads(line)
        user_id = json_data['user']['id']
        user_ids.add(user_id)
        

In [11]:
# setup tweepy
consumer_key = "PQSsqSxGft69D53XGknXya8cL"
consumer_secret = "dlh6EFLADanVNvdl9Q9Lz9Iv3yQQyHY9vxKTS9BN2skflGd45d"
access_token = "1187382763432771584-rwZbZc9lcOcaVdgxcIgqqCPqoQMo3I"
access_token_secret = "nvnIFiZD9L1u6tZwntl24BtsLhKmkBntJ4dpko82fYKg8"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True) # this param makes it not have issues with API limits


In [12]:
# make batches of size 100 of tweets to look up, cuz that's the max per request
batches = []
start = 0
stop = 100
while stop < len(tweet_ids):
    batches.append(tweet_ids[start:stop])
    start += 100
    stop += 100
batches.append(tweet_ids[start:])
    

In [13]:
# look up tweets and save JSONs for tweets authored by users in our dataset

num_found = 0
user_tweet_counts = dict.fromkeys(user_ids,0) # to track user hist length

# for each batch save all tweet JSONs from a user in the dataset
for i, batch in enumerate(batches):
    print('Processing batch #', i)
    try:
        tweet_jsons = api.statuses_lookup(batch)
        
        # filter tweets
        for tweet_json in tweet_jsons:
            user_id = tweet_json.author.id
            
            # save JSON to file if it is from a user in the dataset
            if user_id in user_ids:
                with open(OUTFILE, 'a') as fout:
                    fout.write(json.dumps(tweet_json._json)+'\n') #format json
                    num_found +=1
                    user_tweet_counts[user_id] += 1
                    print('Found! Count is', num_found)
                    
    # handle timeouts (these seem to be rare, Chloe knows more)
    except (Timeout, ConnectionError, IOError, TweepError, TimeoutError) as exc:
        with open(ERROR_FILE, 'a') as fout:
            fout.write('Got stuck on batch # ' +  i + '. Sleeping for 15 seconds...\n')
            fout.write('Error is: ' + str(exc) + '\n')    
        # If a timeout/disconnect error occurs, sleep for 15 secs then continue
        time.sleep(15)
        continue
        

Processing batch # 0
Processing batch # 1
Processing batch # 2
Processing batch # 3
Processing batch # 4
Processing batch # 5
Processing batch # 6
Found! Count is 1
Processing batch # 7
Processing batch # 8
Found! Count is 2
Processing batch # 9
Found! Count is 3
Processing batch # 10
Processing batch # 11
Processing batch # 12
Processing batch # 13
Found! Count is 4
Processing batch # 14
Found! Count is 5
Processing batch # 15
Processing batch # 16
Found! Count is 6
Processing batch # 17
Processing batch # 18
Processing batch # 19
Found! Count is 7
Processing batch # 20
Processing batch # 21
Found! Count is 8
Processing batch # 22
Processing batch # 23
Processing batch # 24
Processing batch # 25
Processing batch # 26
Processing batch # 27
Processing batch # 28
Processing batch # 29
Processing batch # 30
Processing batch # 31
Processing batch # 32
Found! Count is 9
Processing batch # 33
Processing batch # 34
Processing batch # 35
Processing batch # 36
Processing batch # 37
Processing b

In [16]:
# get history length info (key will be length, val is number of users with that length)
freq = {}
for val in user_tweet_counts.values():
    freq[val] = freq.get(val, 0) + 1
freq


{5: 148,
 26: 11,
 1: 1113,
 0: 415,
 15: 26,
 3: 249,
 125: 1,
 2: 418,
 152: 1,
 10: 42,
 8: 71,
 4: 177,
 6: 104,
 12: 31,
 7: 82,
 88: 1,
 13: 33,
 16: 17,
 53: 1,
 9: 49,
 44: 5,
 31: 9,
 111: 1,
 20: 18,
 18: 11,
 163: 2,
 14: 20,
 11: 38,
 107: 1,
 40: 2,
 32: 7,
 23: 11,
 97: 2,
 47: 6,
 17: 18,
 76: 3,
 21: 10,
 27: 2,
 19: 14,
 66: 2,
 64: 4,
 22: 8,
 38: 5,
 43: 2,
 24: 7,
 52: 5,
 30: 3,
 59: 1,
 29: 5,
 69: 1,
 357: 1,
 25: 9,
 50: 3,
 108: 2,
 37: 7,
 42: 3,
 305: 1,
 33: 5,
 126: 2,
 67: 1,
 180: 1,
 58: 1,
 101: 1,
 112: 1,
 39: 4,
 96: 1,
 99: 1,
 34: 5,
 154: 1,
 72: 1,
 60: 1,
 28: 3,
 146: 2,
 86: 2,
 133: 1,
 56: 1,
 45: 3,
 62: 1,
 177: 1,
 110: 1,
 35: 4,
 55: 1,
 57: 1,
 54: 2,
 116: 1,
 49: 1,
 92: 1,
 77: 2,
 87: 2,
 209: 1,
 41: 1,
 51: 3,
 46: 2,
 84: 1,
 83: 2,
 36: 3,
 80: 1,
 226: 1,
 93: 2,
 63: 2,
 149: 1,
 73: 1,
 100: 1,
 253: 1,
 70: 1,
 79: 1}